Setup & Konfiguration

In [25]:
!pip install rdflib pyvis


[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [26]:
# --- Notebook-Header: Pfade, Imports, Namespaces, Helpers ---

from pathlib import Path
import pandas as pd
import json, subprocess, hashlib, sys, math
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS
import networkx as nx

# ----------------------- Pfade anpassen -----------------------
DROID_CSV = Path("/home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_DROIDresults.csv")
EXISTING_RDF_PATH = Path("/home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_DROIDresults.ttl")   # <--- falls vorhanden (z. B. deine Kopie2.txt als .ttl)
OUTPUT_RDF_PATH   = Path("/home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_catalog_prov.ttl")   # <--- neue (erweiterte) Fassung
EXIFTOOL = "/home/renku/work/exiftool/exiftool"  # ggf. absoluter Pfad, z.B. "/home/renku/work/exiftool/exiftool"

# ----------------------- Namespaces ---------------------------
DCA      = Namespace("http://dca.ethz.ch/ontology#")
DCA_ID   = Namespace("http://dca.ethz.ch/id/")
DCA_TECH = Namespace("http://dca.ethz.ch/tech#")
PREMIS   = Namespace("http://www.loc.gov/premis/rdf/v3/")
RICO     = Namespace("https://www.ica.org/standards/RiC/ontology#")
XSDNS    = XSD  # Abkürzung
# DCTERMS ist oben importiert

# ----------------------- Ziel: Bild-/Adobe-Typen -------------
IMG_EXT = {
    # klassische Bilder
    "jpg","jpeg","tif","tiff","png","gif","bmp",
    # RAW (optional – je nach Bestand)
    "dng","cr2","nef","arw"
}
ADOBE_EXT = {"psd","psb","ai","indd","pdf"}  # PDF häufig Photoshop/Illustrator-Export

TARGET_EXT = IMG_EXT | ADOBE_EXT

# === ID-Helfer basierend auf DROID CSV: hash_md5[:16] ===
from rdflib import URIRef

DCA_ID_BASE = "http://dca.ethz.ch/id/"

def dca_file_uri_from_md5(md5_hex: str | None) -> URIRef | None:
    """
    Erzeugt dca-id:file_<md5[:16]> aus einem MD5-Hexstring (ohne Leerzeichen).
    Gibt None zurück, wenn md5_hex leer/ungültig ist.
    """
    if not md5_hex or not isinstance(md5_hex, str):
        return None
    md5_hex = md5_hex.strip().lower()
    if len(md5_hex) < 16:
        return None
    short = md5_hex[:16]
    return URIRef(DCA_ID_BASE + f"file_{short}")

def dca_file_uri_from_path(file_path: str) -> URIRef | None:
    """
    Erzeugt dca-id:file_URI basierend auf DROID MD5-Hash (bevorzugt) oder Pfad-Fallback.
    Diese Funktion ersetzt die alte pfad-basierte Implementierung.
    """
    # 1. Versuche MD5-Hash aus DROID CSV zu verwenden
    md5_hash = md5_for_abs_path(file_path)
    if md5_hash:
        return dca_file_uri_from_md5(md5_hash)
    
    # 2. Fallback: Pfad-basierter Hash (für Rückwärtskompatibilität)
    print(f"⚠️  Fallback zu Pfad-Hash für: {file_path}")
    path_hash = hashlib.sha256(file_path.encode('utf-8')).hexdigest()[:16]
    return URIRef(DCA_ID_BASE + f"file_{path_hash}")

def safe_literal_dt(text: str, datatype=XSD.dateTime):
    try:
        return Literal(text, datatype=datatype)
    except Exception:
        return Literal(text)  # fall back

def run_exiftool_json(files: list, fast=False):
    """Rufe exiftool als JSON auf, tolerant gegen Minor Errors, UTF-8 Dateinamen."""
    if not files:
        return []
    cmd = [EXIFTOOL, "-a","-s","-G1","-json", "-charset","filename=UTF8","-m"]
    if fast:
        cmd.insert(1, "-fast")
    tags = [
        "XMP-xmpMM:DocumentID",
        "XMP-xmpMM:InstanceID",
        "XMP-xmpMM:OriginalDocumentID",
        "XMP-xmpMM:DerivedFromDocumentID",
        "XMP-xmpMM:DerivedFromInstanceID",
        "XMP-xmp:CreatorTool",
        "File:FileName",
        "File:Directory",
        "File:FileModifyDate",
    ]
    cmd += tags + files

    res = subprocess.run(cmd, text=True, capture_output=True)
    out = res.stdout.strip()
    try:
        return json.loads(out) if out else []
    except Exception as e:
        print("EXIF JSON parse error:", e, file=sys.stderr)
        return []

def add_identifier_triple(g: Graph, file_uri: URIRef, id_type: str, value: str):
    """Hänge einen PREMIS-Identifier als Blank Node an ein File-Objekt."""
    if not value:
        return
    bn = BNode()
    g.add((file_uri, PREMIS.hasIdentifier, bn))
    g.add((bn, PREMIS.identifierType, Literal(id_type)))
    g.add((bn, PREMIS.identifierValue, Literal(value)))

DROID‑CSV laden & Dateimenge filtern


In [27]:
# --- DROID CSV laden ---
df = pd.read_csv(DROID_CSV)

# Erwartete Spalten u. a.:
# FILE_PATH, NAME, EXT, LAST_MODIFIED, SIZE, PUID, MIME_TYPE, FORMAT_NAME, FORMAT_VERSION
# (bestätigt durch die Kopfzeilen-Analyse)  # Quelle: DROID-CSV-Headeranalyse

# Nur Dateien mit Extension in unserer Zielmenge
df["EXT"] = df["EXT"].astype(str).str.lower()
cand = df[(df["TYPE"] == "File") & (df["EXT"].isin(TARGET_EXT))].copy()

# Absolute Pfade als String
cand["ABS_PATH"] = cand["FILE_PATH"].astype(str)

print("Gesamt:", len(df), " | Kandidaten (Bild/Adobe):", len(cand))
cand.head(3)

Gesamt: 7120  | Kandidaten (Bild/Adobe): 6487


,ID,PARENT_ID,URI,FILE_PATH,NAME,METHOD,STATUS,SIZE,TYPE,EXT,LAST_MODIFIED,EXTENSION_MISMATCH,HASH,FORMAT_COUNT,PUID,MIME_TYPE,FORMAT_NAME,FORMAT_VERSION,ABS_PATH
14,15,12.0,file:/home/renku/work/dcaonnextcloud-500gb/Dig...,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,testtexture.jpg,Signature,Done,81764.0,File,jpg,2006-04-19T21:18:08,False,5e6306f4d763ec031127db689d5a4c2f,1.0,fmt/44,image/jpeg,JPEG File Interchange Format,1.02,/home/renku/work/dcaonnextcloud-500gb/DigitalM...
21,22,9.0,file:/home/renku/work/dcaonnextcloud-500gb/Dig...,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,060419_plan1_v03.pdf,Signature,Done,361828.0,File,pdf,2006-04-20T07:40:23,False,dc56a28ebb85ce00dc6a67883fc9ea5c,1.0,fmt/18,application/pdf,Acrobat PDF 1.4 - Portable Document Format,1.4,/home/renku/work/dcaonnextcloud-500gb/DigitalM...
23,24,12.0,file:/home/renku/work/dcaonnextcloud-500gb/Dig...,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,060418_Schliessvariante_3.33Seite_12Steine_28L...,Signature,Done,9995110.0,File,psd,2006-04-19T21:34:04,False,d3613f6a611335628522d5155f1cc29c,1.0,x-fmt/92,image/vnd.adobe.photoshop,Adobe Photoshop,NaN,/home/renku/work/dcaonnextcloud-500gb/DigitalM...


Neue MD5-basierte File-IDs testen

In [28]:
# Sicherstellen, dass cand["ABS_PATH"] und df["FILE_PATH"] vorhanden sind
# cand kommt aus Zelle 1 (gefilterte Dateien)

# Erstelle Mapping von FILE_PATH zu MD5-Hash für schnelle Lookups
path_to_md5 = (df[["FILE_PATH","HASH"]]
               .dropna(subset=["FILE_PATH","HASH"])
               .drop_duplicates(subset=["FILE_PATH"])
               .set_index("FILE_PATH")["HASH"]
               .to_dict())

def md5_for_abs_path(p: str) -> str | None:
    """
    Gibt den MD5-Hash für einen Dateipfad zurück, falls im DROID CSV vorhanden.
    """
    return path_to_md5.get(p)

print(f"📊 MD5-Hash-Mapping erstellt für {len(path_to_md5)} Dateien")
print(f"   Beispiele: {list(path_to_md5.items())[:3]}")

📊 MD5-Hash-Mapping erstellt für 7038 Dateien
   Beispiele: [('/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/.DS_Store', 'b3e26c13f3344da34b8894cafd76b6ea'), ('/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/.DS_Store', '2163f741d74e43ebcb692c8897f05449'), ('/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_1/060419_plan1_v03.bak', 'de1492cca90347a7db5514930c1e5b51')]


In [29]:
# Test der neuen MD5-basierten File-URIs
print("🧪 Testing neue MD5-basierte File-URI Generierung:\n")

# Teste mit ein paar Beispieldateien
test_files = cand["ABS_PATH"].head(5).tolist()
for file_path in test_files:
    md5_hash = md5_for_abs_path(file_path)
    old_uri = URIRef(DCA_ID_BASE + f"file_{hashlib.sha256(file_path.encode('utf-8')).hexdigest()[:16]}")
    new_uri = dca_file_uri_from_path(file_path)
    
    print(f"📁 Datei: {Path(file_path).name}")
    print(f"   MD5: {md5_hash[:16] if md5_hash else 'None'}")
    print(f"   ALT: {str(old_uri).split('/')[-1]}")
    print(f"   NEU: {str(new_uri).split('/')[-1]}")
    print(f"   ✅ Verschiedene IDs: {str(old_uri) != str(new_uri)}")
    print()

print(f"📈 Statistik:")
print(f"   Dateien mit MD5: {len([p for p in test_files if md5_for_abs_path(p)])}/{len(test_files)}")
print(f"   MD5-basierte URIs: {len([dca_file_uri_from_path(p) for p in test_files if md5_for_abs_path(p)])}")

🧪 Testing neue MD5-basierte File-URI Generierung:

📁 Datei: testtexture.jpg
   MD5: 5e6306f4d763ec03
   ALT: file_ed7d67aadc715ddf
   NEU: file_5e6306f4d763ec03
   ✅ Verschiedene IDs: True

📁 Datei: 060419_plan1_v03.pdf
   MD5: dc56a28ebb85ce00
   ALT: file_b6c4d966ee107d57
   NEU: file_dc56a28ebb85ce00
   ✅ Verschiedene IDs: True

📁 Datei: 060418_Schliessvariante_3.33Seite_12Steine_28Lagen_working_Expression.5.psd
   MD5: d3613f6a61133562
   ALT: file_d8e7a582fbec2c1d
   NEU: file_d3613f6a61133562
   ✅ Verschiedene IDs: True

📁 Datei: 060419_Plan_2.ai
   MD5: c2cae6160089a0bc
   ALT: file_ad87dffbed3af863
   NEU: file_c2cae6160089a0bc
   ✅ Verschiedene IDs: True

📁 Datei: 060419_Plan_2.pdf
   MD5: 71caab4b9eb51974
   ALT: file_358338c608dfb982
   NEU: file_71caab4b9eb51974
   ✅ Verschiedene IDs: True

📈 Statistik:
   Dateien mit MD5: 5/5
   MD5-basierte URIs: 5


## ✨ Neue MD5-basierte File-IDs

**Wichtige Änderung**: Anstatt Pfad-basierter Hashes verwenden wir jetzt die MD5-Hashes aus dem DROID CSV:

### Vorteile:
- **Konsistent**: Gleiche Datei = Gleiche ID (unabhängig vom Pfad)  
- **Plattform-unabhängig**: Windows vs Unix Pfade irrelevant
- **Inhalt-basiert**: Nur Dateiänderungen erzeugen neue IDs
- **DROID-integriert**: Nutzt bereits berechnete Hashes

### Implementierung:
```python
# ALT: Pfad → SHA256[:16]
old_id = "file_" + hashlib.sha256(path.encode()).hexdigest()[:16]

# NEU: DROID MD5[:16] 
new_id = "file_" + droid_record.hash_md5[:16]
```

EXIF/XMP an einer Beispiel‑PSD auslesen (inkl. IDs)

In [30]:
# Eine Beispiel-PSD aus dem gefilterten DataFrame
ex_psd = cand[cand["EXT"] == "psd"].head(1)
if ex_psd.empty:
    print("Keine PSD gefunden – bitte eine PSD im Bestand wählen.")
else:
    psd_path = ex_psd["ABS_PATH"].iloc[0]
    data = run_exiftool_json([psd_path])
    print("Datei:", psd_path)
    print(json.dumps(data, indent=2))
    # Interpretation für Einsteiger (kurze Erklärung):
    print("\nHinweis:")
    print("- XMP-xmpMM:DocumentID      = Stabiler Dokument-Identifier")
    print("- XMP-xmpMM:InstanceID      = Jede Revision/Instanz hat eine neue ID")
    print("- XMP-xmpMM:DerivedFrom*    = Verknüpft auf Quelle/Parent (Ableitung)")

Datei: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2/060418_Schliessvariante_3.33Seite_12Steine_28Lagen_working_Expression.5.psd
[
  {
    "SourceFile": "/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2/060418_Schliessvariante_3.33Seite_12Steine_28Lagen_working_Expression.5.psd",
    "ExifTool:ExifToolVersion": 13.51,
    "System:FileName": "060418_Schliessvariante_3.33Seite_12Steine_28Lagen_working_Expression.5.psd",
    "System:Directory": "/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2",
    "System:FileSize": "10.0 MB",
    "System:FileModifyDate": "2006:04:19 21:34:04+00:00",
    "

EXIF/XMP an einer Beispiel‑JPG auslesen (inkl. IDs)

In [31]:
# Eine Beispiel-JPG
ex_jpg = cand[cand["EXT"].isin(["jpg","jpeg"])].head(1)
if ex_jpg.empty:
    print("Keine JPG/JPEG gefunden – bitte ein Beispiel wählen.")
else:
    jpg_path = ex_jpg["ABS_PATH"].iloc[0]
    data = run_exiftool_json([jpg_path])
    print("Datei:", jpg_path)
    print(json.dumps(data, indent=2))
    print("\nHinweis:")
    print("- Bei JPG sind XMP-IDs nicht immer gesetzt; wenn vorhanden, nutzen wir sie wie bei PSD.")

Datei: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2/testtexture.jpg
[
  {
    "SourceFile": "/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2/testtexture.jpg",
    "ExifTool:ExifToolVersion": 13.51,
    "System:FileName": "testtexture.jpg",
    "System:Directory": "/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2",
    "System:FileSize": "82 kB",
    "System:FileModifyDate": "2006:04:19 21:18:08+00:00",
    "System:FileAccessDate": "2006:04:19 21:18:08+00:00",
    "System:FileInodeChangeDate": "2006:04:19 21:18:08+00:00",
    "System:FilePermissions": "-rw-r--r--",
    "File:FileType": "

Batch‑Auslesen (XMP‑IDs) & Index für Derivatsuche aufbauen

In [32]:
# Wir lesen XMP für alle Kandidaten in Batches, speichern kompakt die relevanten Felder
BATCH = 100
records = []

paths = cand["ABS_PATH"].tolist()
total = len(paths)
for i in range(0, total, BATCH):
    batch = paths[i:i+BATCH]
    js = run_exiftool_json(batch)
    for row in js:
        rec = {
            "SourceFile": row.get("SourceFile"),
            "DocumentID": row.get("XMP-xmpMM:DocumentID"),
            "InstanceID": row.get("XMP-xmpMM:InstanceID"),
            "OriginalDocumentID": row.get("XMP-xmpMM:OriginalDocumentID"),
            "DerivedFromDocumentID": row.get("XMP-xmpMM:DerivedFromDocumentID"),
            "DerivedFromInstanceID": row.get("XMP-xmpMM:DerivedFromInstanceID"),
            "FileModifyDate": row.get("File:FileModifyDate"),
        }
        records.append(rec)
    print(f"{min(i+BATCH, total)}/{total}")

xmp_df = pd.DataFrame(records)
print("Gelesen:", len(xmp_df))

# Indexe zum späteren Matchen aufbauen:
# - nach DocumentID und nach InstanceID
id_index_doc  = (xmp_df[~xmp_df["DocumentID"].isna()]
                 .drop_duplicates(subset=["DocumentID"])
                 .set_index("DocumentID")["SourceFile"].to_dict())
id_index_inst = (xmp_df[~xmp_df["InstanceID"].isna()]
                 .drop_duplicates(subset=["InstanceID"])
                 .set_index("InstanceID")["SourceFile"].to_dict())

xmp_df.head(3)

100/6487
200/6487
300/6487
400/6487
500/6487
600/6487
700/6487
800/6487
900/6487
1000/6487
1100/6487
1200/6487
1300/6487
1400/6487
1500/6487
1600/6487
1700/6487
1800/6487
1900/6487
2000/6487
2100/6487
2200/6487
2300/6487
2400/6487
2500/6487
2600/6487
2700/6487
2800/6487
2900/6487
3000/6487
3100/6487
3200/6487
3300/6487
3400/6487
3500/6487
3600/6487
3700/6487
3800/6487
3900/6487
4000/6487
4100/6487
4200/6487
4300/6487
4400/6487
4500/6487
4600/6487
4700/6487
4800/6487
4900/6487
5000/6487
5100/6487
5200/6487
5300/6487
5400/6487
5500/6487
5600/6487
5700/6487
5800/6487
5900/6487
6000/6487
6100/6487
6200/6487
6300/6487
6400/6487
6487/6487
Gelesen: 6487


,SourceFile,DocumentID,InstanceID,OriginalDocumentID,DerivedFromDocumentID,DerivedFromInstanceID,FileModifyDate
0,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,uuid:D9170E74C1CFDA118C30CF8EA9BFE8B1,uuid:A53BA05DE9CFDA118C30CF8EA9BFE8B1,None,uuid:18FD04FBB2CFDA118C30CF8EA9BFE8B1,uuid:B0B29CD2B8CFDA118C30CF8EA9BFE8B1,None
1,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,uuid:12A3774BF0CFDA1186109B3BE9744C33,uuid:53c3d276-af86-430f-b52e-796f0c6a18b8,None,uuid:38614f61-7afe-4454-81d2-8b17e2df8c8f,uuid:07d991a4-40b9-46aa-bebf-1070723b599b,None
2,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,uuid:C74E838EEACFDA118C30CF8EA9BFE8B1,uuid:C84E838EEACFDA118C30CF8EA9BFE8B1,None,uuid:C54E838EEACFDA118C30CF8EA9BFE8B1,uuid:C54E838EEACFDA118C30CF8EA9BFE8B1,None


Bestehendes RDF laden & Graph vorbereiten

In [33]:
# RDF laden (falls vorhanden), sonst leerer Graph
g = Graph()
g.bind("dca", DCA)
g.bind("dca-id", DCA_ID)
g.bind("dca-tech", DCA_TECH)
g.bind("dcterms", DCTERMS)
g.bind("premis", PREMIS)
g.bind("rico", RICO)
g.bind("xsd", XSDNS)

if EXISTING_RDF_PATH.exists():
    g.parse(EXISTING_RDF_PATH, format="turtle")
    print("Bestehendes RDF geladen:", EXISTING_RDF_PATH)
else:
    print("Kein bestehendes RDF gefunden – starte mit leerem Graphen.")

print("Tripel (vorher):", len(g))

Bestehendes RDF geladen: /home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_DROIDresults.ttl
Tripel (vorher): 94173


XMP‑Identifier als PREMIS‑Identifier anreichern

In [34]:
# Wir hängen XMP-IDs als PREMIS-Identifier an jede Datei (falls vorhanden).
# ✨ NEU: Verbindung über dca-id:file_<md5[:16]> statt Pfad-Hash

added_ids = 0
md5_based_uris = 0
path_based_uris = 0

for _, row in xmp_df.iterrows():
    abs_path = row["SourceFile"]
    
    # ✨ Neue MD5-basierte URI-Generierung
    file_uri = dca_file_uri_from_path(abs_path)
    
    # Statistik sammeln
    if md5_for_abs_path(abs_path):
        md5_based_uris += 1
    else:
        path_based_uris += 1

    # Sicherstellen, dass Objekt typisiert ist (idR schon vorhanden – doppelte Tripel sind ok)
    g.add((file_uri, RDF.type, DCA.ArchiveFile))
    g.add((file_uri, RDF.type, PREMIS.Object))
    g.add((file_uri, RDF.type, RICO.Record))

    # XMP IDs als PREMIS Identifier
    if row.get("DocumentID"):
        add_identifier_triple(g, file_uri, "XMP DocumentID", row["DocumentID"]); added_ids += 1
    if row.get("InstanceID"):
        add_identifier_triple(g, file_uri, "XMP InstanceID", row["InstanceID"]); added_ids += 1
    if row.get("OriginalDocumentID"):
        add_identifier_triple(g, file_uri, "XMP OriginalDocumentID", row["OriginalDocumentID"]); added_ids += 1

print("✨ Neue ID-Methode erfolgreich verwendet:")
print(f"   MD5-basierte URIs: {md5_based_uris}")
print(f"   Pfad-basierte URIs (Fallback): {path_based_uris}")
print(f"   Identifier-Knoten hinzugefügt: {added_ids}")
print(f"   Tripel (jetzt): {len(g)}")

✨ Neue ID-Methode erfolgreich verwendet:
   MD5-basierte URIs: 6487
   Pfad-basierte URIs (Fallback): 0
   Identifier-Knoten hinzugefügt: 3543
   Tripel (jetzt): 124110


Derivations‑Beziehungen (PREMIS) aus XMP:DerivedFrom ableiten

In [35]:
added_rel = 0
unresolved = []

for _, row in xmp_df.iterrows():
    child_path = row["SourceFile"]
    child_uri  = dca_file_uri_from_path(child_path)

    parent_path = None
    # 1) Match via DerivedFromDocumentID
    if row.get("DerivedFromDocumentID") and row["DerivedFromDocumentID"] in id_index_doc:
        parent_path = id_index_doc[row["DerivedFromDocumentID"]]
    # 2) sonst via DerivedFromInstanceID
    elif row.get("DerivedFromInstanceID") and row["DerivedFromInstanceID"] in id_index_inst:
        parent_path = id_index_inst[row["DerivedFromInstanceID"]]

    if parent_path:
        parent_uri = dca_file_uri_from_path(parent_path)
        # Tripel hinzufügen (beide Richtungen)
        g.add((child_uri, PREMIS.hasSource, parent_uri))
        g.add((parent_uri, PREMIS.isSourceOf, child_uri))
        added_rel += 1
    else:
        # notieren, wenn eine Ableitung deklariert ist, aber wir kein Gegenstück finden
        if row.get("DerivedFromDocumentID") or row.get("DerivedFromInstanceID"):
            unresolved.append({
                "child": child_path,
                "FromDocID": row.get("DerivedFromDocumentID"),
                "FromInstID": row.get("DerivedFromInstanceID"),
            })

print("PREMIS-Ableitungen erzeugt:", added_rel)
print("Nicht auflösbare Ableitungen:", len(unresolved))
unresolved[:3]

PREMIS-Ableitungen erzeugt: 32
Nicht auflösbare Ableitungen: 27


[{'child': '/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2/testtexture.jpg',
  'FromDocID': 'uuid:18FD04FBB2CFDA118C30CF8EA9BFE8B1',
  'FromInstID': 'uuid:B0B29CD2B8CFDA118C30CF8EA9BFE8B1'},
 {'child': '/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_1/060419_plan1_v03.pdf',
  'FromDocID': 'uuid:38614f61-7afe-4454-81d2-8b17e2df8c8f',
  'FromInstID': 'uuid:07d991a4-40b9-46aa-bebf-1070723b599b'},
 {'child': '/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2/060418_Schliessvariante_3.33Seite_12Steine_28Lagen_working_Expression.5.psd',
  'FromDocID': 'uuid:C54E838EEACFDA118C30CF8EA9BFE8B1',
  'F

RDF serialisieren (ergänzte Fassung)

In [36]:
g.serialize(destination=str(OUTPUT_RDF_PATH), format="turtle")
print("✔ RDF gespeichert:", OUTPUT_RDF_PATH)
print("Tripel gesamt:", len(g))

✔ RDF gespeichert: /home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_catalog_prov.ttl
Tripel gesamt: 124174


SPARQL‑Beispiele (im Graphen) & einfache Visualisierung

In [37]:
# 1) Alle Derivationen (Child -> Parent)
q1 = """
PREFIX premis: <http://www.loc.gov/premis/rdf/v3/>
SELECT ?child ?parent WHERE {
  ?child premis:hasSource ?parent .
} LIMIT 50
"""
for row in g.query(q1):
    print(row.child, " <--derived-- ", row.parent)

# 2) Alle Objekte mit XMP DocumentID
q2 = """
PREFIX premis: <http://www.loc.gov/premis/rdf/v3/>
SELECT ?obj ?val WHERE {
  ?obj premis:hasIdentifier ?id .
  ?id premis:identifierType "XMP DocumentID" ;
      premis:identifierValue ?val .
} LIMIT 50
"""
list(g.query(q2))[:5]

http://dca.ethz.ch/id/file_5f42b3d9c6c3260c  <--derived--  http://dca.ethz.ch/id/file_661681d5c1948aaf
http://dca.ethz.ch/id/file_43950ae0719a9cdb  <--derived--  http://dca.ethz.ch/id/file_dc3e2b447a5dd74b
http://dca.ethz.ch/id/file_0c5c5ffd1ba83473  <--derived--  http://dca.ethz.ch/id/file_d3a08307fba199b7
http://dca.ethz.ch/id/file_60c2aca3a3d2a6b6  <--derived--  http://dca.ethz.ch/id/file_7f5dda278efd4da7
http://dca.ethz.ch/id/file_67ada385dc10b1f3  <--derived--  http://dca.ethz.ch/id/file_60c2aca3a3d2a6b6
http://dca.ethz.ch/id/file_8d331588a3276854  <--derived--  http://dca.ethz.ch/id/file_67ada385dc10b1f3
http://dca.ethz.ch/id/file_ad1942c6a755ce2d  <--derived--  http://dca.ethz.ch/id/file_67ada385dc10b1f3
http://dca.ethz.ch/id/file_3c9968b7ca5ed88a  <--derived--  http://dca.ethz.ch/id/file_f893cf140e47c312
http://dca.ethz.ch/id/file_f893cf140e47c312  <--derived--  http://dca.ethz.ch/id/file_7b0540e8799641eb
http://dca.ethz.ch/id/file_d527dc13dc5f04bb  <--derived--  http://dca.eth

[(rdflib.term.URIRef('http://dca.ethz.ch/id/file_5e6306f4d763ec03'),
  rdflib.term.Literal('uuid:D9170E74C1CFDA118C30CF8EA9BFE8B1')),
 (rdflib.term.URIRef('http://dca.ethz.ch/id/file_dc56a28ebb85ce00'),
  rdflib.term.Literal('uuid:12A3774BF0CFDA1186109B3BE9744C33')),
 (rdflib.term.URIRef('http://dca.ethz.ch/id/file_d3613f6a61133562'),
  rdflib.term.Literal('uuid:C74E838EEACFDA118C30CF8EA9BFE8B1')),
 (rdflib.term.URIRef('http://dca.ethz.ch/id/file_c2cae6160089a0bc'),
  rdflib.term.Literal('uuid:83FA0ACCECCFDA118F56F7E076900BAC')),
 (rdflib.term.URIRef('http://dca.ethz.ch/id/file_71caab4b9eb51974'),
  rdflib.term.Literal('uuid:96dab556-46bb-46d6-9077-e5f9d99616c4'))]

Mapping RDF‑URI → Dateiname/Tooltip

In [38]:
from pathlib import Path

# --- 1) Map aus XMP-Ergebnissen (bevorzugt, weil garantiert aktueller Pfad) ---
uri_to_label = {}
uri_to_title = {}

if not xmp_df.empty:
    for _, r in xmp_df.iterrows():
        src = r.get("SourceFile")
        if not src:
            continue
        file_uri = dca_file_uri_from_path(src)
        uri_to_label[str(file_uri)] = Path(src).name                # z.B. "testtexture.jpg"
        uri_to_title[str(file_uri)] = src                           # Tooltip: voller Pfad

# --- 2) Fallback: aus DROID-CSV (cand) ---
# cand enthält: ABS_PATH (voll), NAME (Dateiname), FILE_PATH (voll)
for _, r in cand.iterrows():
    abs_path = r["ABS_PATH"]
    file_uri = dca_file_uri_from_path(abs_path)
    uri_s = str(file_uri)
    if uri_s not in uri_to_label:
        uri_to_label[uri_s] = r.get("NAME", Path(abs_path).name)    # "NAME" bevorzugt, sonst aus Pfad
    if uri_s not in uri_to_title:
        uri_to_title[uri_s] = abs_path

# --- 3) Sicherheits-Fallback (sollte kaum noch greifen) ---
def label_for(uri: str) -> str:
    return uri_to_label.get(uri, uri.split("/")[-1])  # allerletztes Segment der URI

def title_for(uri: str) -> str:
    return uri_to_title.get(uri, uri)                 # Tooltip im Zweifel die URI selbst

Bonus: Graphische Darstellung der Ableitungs‑Kanten (PREMIS)

In [39]:
# --- Bonus: Graphische Darstellung der Ableitungs-Kanten (PREMIS) mit Dateinamen-Labels ---
import networkx as nx

# DiGraph Parent -> Child (isSourceOf-Richtung)
G = nx.DiGraph()
for (s, p, o) in g.triples((None, PREMIS.hasSource, None)):
    # s = child, o = parent; Kante Parent -> Child
    G.add_edge(str(o), str(s))

print("Knoten:", G.number_of_nodes(), "Kanten:", G.number_of_edges())

# Versuche pyvis für HTML-Visualisierung
try:
    from pyvis.network import Network
    net = Network(height="700px", width="100%", directed=True, notebook=True)
    net.toggle_physics(True)

    # Knoten mit Dateinamen als Label + vollem Pfad als Tooltip
    for n in G.nodes:
        net.add_node(
            n,
            label=label_for(n),          # z. B. "testtexture.jpg"
            title=title_for(n),          # Tooltip: absoluter Pfad
            shape="dot",
            size=12
        )

    # Kanten mit sprechender Beschriftung
    for u, v in G.edges:
        net.add_edge(u, v, title="premis:isSourceOf / premis:hasSource", arrows="to")

    net.show("derivations_graph.html")
    print("✔ Visualisierung gespeichert: derivations_graph.html")
except Exception as e:
    print("Hinweis: Für HTML-Graph bitte 'pyvis' installieren. Fehler:", e)

Knoten: 49 Kanten: 32
derivations_graph.html
✔ Visualisierung gespeichert: derivations_graph.html
